In [ ]:
""" 
so by using openai and langchain can i make an ats friendly application for that whay packages do i need to install guide me step by step
Features
- by using pypdfreader read the pdf
- and devide that in to small chunks
- and convert that into vectors either job description or resume chunks
- compare the  with the job description with resume chunks viseversa
- fillnally results the resume score, with suggestions to improve that needs in the resume
    and gramitical mistake fixing if any other things needs to achive please do that

if i miss any other content or any other implementation please explain that and implement the process

list the packages what do i need to install
i am using the conda notebook
so must the code for notebook with langchain and openai
use the all the openai features and must need the good accurated output from the openai


"""

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os
import language_tool_python

In [3]:
load_dotenv()

True

In [4]:
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

In [5]:
def load_pdf_text(pdf_path):
    loader=PyPDFLoader(pdf_path)
    page=loader.load()
    return " ".join([p.page_content for p in page])

In [6]:
pdf_path='./pdfs/Niranjan_Raju_Resume.pdf'

In [7]:
resume_text=load_pdf_text(pdf_path)

In [8]:
jd_text="""
Job Title: Full Stack Developer

Location: Hyderabad
Position Type: Full-Time (Work from Office Only)
Experience: 1 to 2  Years

Role Overview:
We are looking for a proactive and skilled Full Stack Developer with 8 to 18 months of experience to join our engineering team. This role is perfect for developers who are excited to work across the stack, contribute to building scalable systems, and experiment with AI and automation technologies.

Key Responsibilities:

Develop and maintain scalable web applications using React.js and Node.js
Write clean, maintainable code with a focus on performance and security
Support integration of Python-based AI/ML modules (bonus)
Required Skills & Qualifications:

Strong foundation in JavaScript/TypeScript for both frontend and backend
Experience with React.js and Node.js
Familiarity with relational and NoSQL databases (e.g., PostgreSQL, DynamoDB)
Understanding of cloud services and serverless architecture (AWS Lambda, Kinesis)
Basic exposure to Python for automation or ML is a plus
A growth mindset, team player attitude, and willingness to learn new technologies
Good to Have

Exposure to CI/CD workflows
Knowledge of RESTful API design and microservices
Understanding of basic AI/ML concepts or pipelines
"""

In [9]:
def text_splitter(text):
    splitter=RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50
    )
    chunk=splitter.split_text(text)
    return chunk 

In [10]:
resume_chunk=text_splitter(resume_text)
jd_chunk=text_splitter(jd_text)

In [11]:
embeddings=OpenAIEmbeddings()

In [12]:
resume_vectorStore=FAISS.from_texts(resume_chunk, embeddings)
jd_vectorStore=FAISS.from_texts(jd_chunk, embeddings)

In [13]:
def similarity_score(query, vectorStore):
    docs_and_score=vectorStore.similarity_search_with_score(query, k=3)
    avg_score=sum([score for _, score in docs_and_score]) / len(docs_and_score)
    return 1 /(1 + avg_score)

In [14]:
jd_to_resume_score=sum(
    similarity_score(chunk, resume_vectorStore) for chunk in jd_chunk ) / len(jd_chunk)

In [15]:
resume_to_jd_score = sum(
    similarity_score(chunk, jd_vectorStore) for chunk in resume_chunk) / len(resume_chunk)

In [16]:
final_score = round((jd_to_resume_score + resume_to_jd_score) / 2 * 100, 2)


In [17]:
print(f"Resume Match Score: {final_score}%")

Resume Match Score: 72.58000183105469%


In [18]:
tool = language_tool_python.LanguageTool('en-US')
matches = tool.check(resume_text)
print(f"Found {len(matches)} grammar issues.")

Found 24 grammar issues.


In [19]:
llm=ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

In [20]:
template = """
You are an ATS (Applicant Tracking System) resume optimization expert.

A user has uploaded the following resume:

{resume}

And the job description is:

{jd}

Based on a detailed comparison:

- Suggest improvements to align the resume with the job description.
- Identify missing keywords, skills, or relevant experiences.
- Provide a resume optimization score out of 100.
- Correct any grammar or formatting issues in the resume.

Return the response in **markdown format**, organized into the following sections:

### Resume Improvement Suggestions
...

### Missing Keywords or Experience
...

### Grammar and Formatting Corrections
...

### Resume Optimization Score
...

"""


In [21]:
prompt=PromptTemplate.from_template(template)

In [22]:
chain = prompt | llm


In [23]:
res=chain.invoke({"resume":resume_text, "jd":jd_text})

In [24]:
print(res.content)

### Resume Improvement Suggestions

1. **Highlight Relevant Experience**: Emphasize your experience with React.js and Node.js, as these are crucial for the job. Consider restructuring the work experience section to highlight these technologies prominently.

2. **Include Python Exposure**: If you have any experience with Python, even if minimal, include it to align with the job's preference for Python exposure.

3. **Cloud Services and Serverless Architecture**: If you have any experience or knowledge in cloud services like AWS or serverless architecture, mention it. This aligns with the job's requirements.

4. **CI/CD Workflows**: If you have experience with CI/CD, include it in your resume. This is listed as a "Good to Have" skill in the job description.

5. **AI/ML Concepts**: If you have any exposure to AI/ML, even if it's through personal projects or learning, mention it to align with the job's preferences.

6. **Reorder Sections**: Consider placing the "Technologies" section after